In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv
import tensorflow as tf
                    
# load model
age_model = tf.keras.models.load_model('../data/age_model_16_11_2021.h5')
gender_model = tf.keras.models.load_model('../data/gender_model_16_2021.h5')

2021-11-18 11:23:15.430537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 11:23:15.435577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 11:23:15.435920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 11:23:15.437378: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
classes = ['F','M']
decoding = {0:'0-2', 1:'4-6', 2:'8-13',3:'15-20',4:'25-32',5:'38-43',6:'48-53',7:'60+'}

image_path = '../data/friends.jpeg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image)
# apply face detection
face, confidence = cv.detect_face(image)


[[[122 133 153]
  [121 132 152]
  [121 132 152]
  ...
  [206 223 239]
  [206 223 239]
  [206 223 239]]

 [[123 134 154]
  [122 133 153]
  [121 132 152]
  ...
  [206 223 239]
  [206 223 239]
  [206 223 239]]

 [[123 134 154]
  [122 133 153]
  [122 133 153]
  ...
  [207 224 240]
  [206 223 239]
  [206 223 239]]

 ...

 [[ 79  92 109]
  [ 80  93 110]
  [ 80  93 110]
  ...
  [152  13  52]
  [150  11  50]
  [145   6  45]]

 [[ 74  87 104]
  [ 76  89 106]
  [ 79  92 109]
  ...
  [149  15  52]
  [145  11  48]
  [138   6  43]]

 [[ 70  83 100]
  [ 73  86 103]
  [ 78  91 108]
  ...
  [146  14  51]
  [143  11  48]
  [136   6  42]]]


In [6]:
print(face)
# loop through detected faces
for idx, f in enumerate(face):
    # get corner points of face rectangle        
    (startX, startY) = f[0], f[1]
    (endX, endY) = f[2], f[3]

    rect = cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)
    face_crop = np.copy(image[startY:endY,startX:endX])
    face_crop= cv2.resize(face_crop, (128,128))
    face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

    age_prediction = age_model.predict(face_crop) 
    
    #decoding 
    index = np.argmax(age_prediction)
    decoding = {0:'0-2', 1:'4-6', 2:'8-13',3:'15-20',4:'25-32',5:'38-43',6:'48-53',7:'60+'}
    
    gender_prediction = gender_model.predict(face_crop)[0] # model.predict return a 2D matrix, ex: [[9.9993384e-01 7.4850512e-05]]
    gen = "M" if gender_prediction[0] > 0.5 else "F"
    label = str(decoding[index])+" "+str(gen)
    
    plt.figure(figsize=(20,20))
    plt.subplot(1,3,1)
    plt.title(f'Prediction: {label}')
    plt.imshow(image)
    plt.imshow(rect)
    plt.subplot(1,3,2)
    plt.title('imagen recortada')
    plt.imshow(face_crop)
    # apply gender detection on face


[[1088, 227, 1288, 512], [839, 199, 926, 322], [136, 355, 261, 500], [28, 288, 127, 397], [579, 198, 652, 292], [393, 206, 463, 293]]


ValueError: in user code:

    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/pj/miniconda3/envs/cv/lib/python3.7/site-packages/keras/engine/input_spec.py", line 263